## Imports

In [1]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.5 MB/s eta 0:00:00


In [2]:
!pip install datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=f2c2379aa72bdac5b164bb23ed85623d6e6edc0c6d01dd4439a4b41e3bb9c096
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
data_drive = "drive/MyDrive/ebay_data_2023"
os.listdir(data_drive)

['Train_Tagged_Titles.tsv',
 'Listing_Titles.tsv',
 'train_data_tokenized.csv',
 'train_data_ffilled.csv',
 '.DS_Store',
 'train_dfBI.csv',
 'models',
 'results',
 'eBay NER strategies.gdoc',
 'notebooks']

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
!pip install distance
import distance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16256 sha256=5a5ee6075411f55d2c3e1119748c4510e3eb6ecd409769f1d8856454ff0da3c0
  Stored in directory: /root/.cache/pip/wheels/e8/bb/de/f71bf63559ea9a921059a5405806f7ff6ed612a9231c4a9309
Successfully built distance


In [7]:
import pandas as pd
import numpy as np
import csv

In [8]:
! pip install pytorch-crf

In [9]:
from transformers import AutoTokenizer
import datasets
from transformers import DataCollatorForTokenClassification
import evaluate
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import get_scheduler
from tensorflow.python.client import device_lib

In [10]:
import torch
import torch.nn as nn
from torchcrf import CRF
from torch.utils.data import DataLoader
from torch.optim import AdamW

## Preprocess Data

In [11]:
train_df = pd.read_csv(data_drive + "/Train_Tagged_Titles.tsv", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
old_tag = train_df.Tag.copy()
train_df

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
...,...,...,...,...
55178,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Sportschuhe,Produktart
55179,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Wanderschuh,NaN
55180,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Big,No Tag
55181,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Size,No Tag


In [12]:
#quiz dataset
test_records = pd.read_csv(data_drive + "/Listing_Titles.tsv", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE).loc[5000:29999,:].reset_index(drop=True)


In [13]:
aspect_names = set(pd.unique(old_tag)) - set([np.nan])
aspect_names, len(aspect_names) # includes "No Tag", "Obscure"

({'Abteilung',
  'Aktivität',
  'Akzente',
  'Anlass',
  'Besonderheiten',
  'Charakter',
  'Charakter Familie',
  'Dämpfungsgrad',
  'EU-Schuhgröße',
  'Erscheinungsjahr',
  'Farbe',
  'Futtermaterial',
  'Gewebeart',
  'Herstellernummer',
  'Herstellungsland und -region',
  'Innensohlenmaterial',
  'Jahreszeit',
  'Laufsohlenmaterial',
  'Marke',
  'Maßeinheit',
  'Modell',
  'Muster',
  'No Tag',
  'Obermaterial',
  'Obscure',
  'Produktart',
  'Produktlinie',
  'Schuhschaft-Typ',
  'Schuhweite',
  'Stil',
  'Stollentyp',
  'Thema',
  'UK-Schuhgröße',
  'US-Schuhgröße',
  'Verschluss',
  'Zwischensohlen-Typ'},
 36)

In [14]:
train_df.Tag = train_df.Tag.fillna("I-" + train_df.Tag.ffill()[train_df.Tag.isna()])
train_df.Tag.loc[train_df.Tag == "I-No Tag"] = "No Tag" # correct for the No Tag (there can be no interior)
train_df.Tag.loc[train_df.Tag == "I-Obscure"] = "Obscure" # correct for the No Tag (there can be no interior)

In [15]:
# train_df.to_csv(data_drive + "/train_dfBI.csv")

In [16]:
pd.unique(train_df.Tag), len(pd.unique(train_df.Tag)) # notice this does not encompass all 70 classes (36*2) - 2 (for No Tag, Obscure)
pass

In [17]:
not_to_include = set(["No Tag", "Obscure"])
asp_arr = np.array(list(aspect_names - not_to_include))
asp_arr.sort()
interior_arr = ("I-"+ pd.Series(list(aspect_names - not_to_include))).sort_values().values
all_aspect_names = np.append(asp_arr, interior_arr)
all_aspect_names = np.append(all_aspect_names, list(not_to_include))
#all aspect names, all interior aspect names, Obscure, No Tag = 70 aspect names

In [18]:
id2label = dict(zip(list(range(len(all_aspect_names))), all_aspect_names))
label2id = dict(zip(all_aspect_names, list(range(len(all_aspect_names)))))

In [19]:
train_df["ner_tags"] = train_df.Tag.apply(lambda val: label2id[val])
train_df["Record Number"] = train_df["Record Number"].astype(int)

## Tokenize

The next step is to load a DistilBERT tokenizer to preprocess the `tokens` field:

In [20]:
lowercased_tokenizer = AutoTokenizer.from_pretrained("numind/entity-recognition-general-sota-v1", truncation=True, do_lower_case=False, add_prefix_space=True)
lowercased_tokenizer.do_lower_case = True

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

As you saw in the example `tokens` field above, it looks like the input has already been tokenized. But the input actually hasn't been tokenized yet and you'll need to set `is_split_into_words=True` to tokenize the words into subwords. For example:

In [21]:
tokenized_input = lowercased_tokenizer("Here is Wolfgang sentence".lower())
tokens = lowercased_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['<s>', 'Ġhere', 'Ġis', 'Ġwolf', 'gang', 'Ġsentence', '</s>']

In [22]:
tokenized_input = lowercased_tokenizer("Here is Wolfgang sentence")
tokens = lowercased_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
[s.lower() for s in tokens]

['<s>', 'ġhere', 'ġis', 'ġwolfgang', 'ġsentence', '</s>']

In [23]:
train_ds = datasets.Dataset.from_pandas(train_df)
train_ds

Dataset({
    features: ['Record Number', 'Title', 'Token', 'Tag', 'ner_tags'],
    num_rows: 55183
})

In [24]:
example = train_df.loc[train_df["Record Number"] == 1, ["Record Number", "Token", "ner_tags"]].to_dict(orient="list")
example

{'Record Number': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'Token': ['Supreme',
  'Nike',
  'SB',
  'Dunk',
  'High',
  'By',
  'any',
  'Means',
  'Red',
  'US10',
  'EU44',
  'Supreme',
  'Box',
  'Logo',
  'Air',
  'Force'],
 'ner_tags': [20, 18, 24, 58, 25, 20, 54, 54, 10, 31, 8, 69, 69, 2, 24, 58]}

In [25]:
datasets.Dataset.from_dict(example)["ner_tags"]

[20, 18, 24, 58, 25, 20, 54, 54, 10, 31, 8, 69, 69, 2, 24, 58]

In [26]:
def combine(x):
  d = x.to_dict(orient="list")
  d["id"] = d["Record Number"][0]
  del d["Record Number"]
  del d["Title"]
  del d["Tag"]
  d["tokens"] = d.pop("Token")
  d["tokens"] = [s.lower() for s in d["tokens"]]
  return d["ner_tags"], d["tokens"]

In [27]:
pre_ds = pd.DataFrame(train_df.groupby(["Record Number", "Title"]).apply(combine))

In [28]:
pre_ds

,,0
Record Number,Title,
1,Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force,"([20, 18, 24, 58, 25, 20, 54, 54, 10, 31, 8, 6..."
2,New Balance 530 Männer und Frauen Laufschuhe mit Buchstaben N bequeme Laufschuhe,"([18, 52, 20, 0, 69, 0, 23, 69, 21, 55, 69, 23..."
3,♥ MICHAEL KORS Sneaker Gr 39 ♥,"([69, 18, 52, 27, 69, 8, 69], [♥, michael, kor..."
4,New Balance ML 574 EGO Turnschuhe grün,"([18, 52, 20, 54, 54, 23, 10], [new, balance, ..."
5,"Nike air jordan 9 og , space jam , hare , bugs bunny , EU 45 / US 11 NEU + RECHNUNG","([18, 24, 58, 20, 54, 69, 10, 20, 69, 10, 69, ..."
...,...,...
4996,Adidas Raf Simons Sneakers,"([18, 24, 58, 27], [adidas, raf, simons, sneak..."
4997,"Ludwig Reiter Andalusier , Nubuk navy Schuhe Damen Gr . 39 NEU","([18, 52, 20, 69, 22, 10, 23, 0, 69, 69, 8, 69..."
4998,Adidas Superstar Laceless / Adidas Originals / Adidas Sneaker / weiß / schwarz / FV3017,"([18, 24, 32, 69, 18, 24, 69, 18, 27, 69, 10, ..."


In [29]:
pre_ds[['ner_tags', 'tokens']] = pd.DataFrame(pre_ds[0].tolist(), index=pre_ds.index)
pre_ds = pre_ds.reset_index().loc[:, ["Record Number", "tokens", "ner_tags"]].rename(columns={"Record Number":"id"})

In [30]:
ebay_ds = datasets.Dataset.from_pandas(pre_ds)
ebay_ds

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 5000
})

In [32]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = lowercased_tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to 70.
            if word_idx is None:
                label_ids.append(70)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(70)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [33]:
id2label[70] = 'Special'
label2id['Special'] = 70

In [34]:
tokenized_ebay = ebay_ds.map(tokenize_and_align_labels, batched=True)
tokenized_ebay

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [35]:
x=list(zip(lowercased_tokenizer.convert_ids_to_tokens(tokenized_ebay["input_ids"][0]), [id2label[l] for l in tokenized_ebay["labels"][0]], tokenized_ebay["attention_mask"][0]))
x

[('<s>', 'Special', 1),
 ('Ġsupreme', 'Modell', 1),
 ('Ġn', 'Marke', 1),
 ('ike', 'Special', 1),
 ('Ġs', 'Produktlinie', 1),
 ('b', 'Special', 1),
 ('Ġdunk', 'I-Produktlinie', 1),
 ('Ġhigh', 'Schuhschaft-Typ', 1),
 ('Ġby', 'Modell', 1),
 ('Ġany', 'I-Modell', 1),
 ('Ġmeans', 'I-Modell', 1),
 ('Ġred', 'Farbe', 1),
 ('Ġus', 'US-Schuhgröße', 1),
 ('10', 'Special', 1),
 ('Ġe', 'EU-Schuhgröße', 1),
 ('u', 'Special', 1),
 ('44', 'Special', 1),
 ('Ġsupreme', 'No Tag', 1),
 ('Ġbox', 'No Tag', 1),
 ('Ġlogo', 'Akzente', 1),
 ('Ġair', 'Produktlinie', 1),
 ('Ġforce', 'I-Produktlinie', 1),
 ('</s>', 'Special', 1)]

In [36]:
tokenized_ebay = tokenized_ebay.remove_columns(ebay_ds.column_names)

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [37]:
data_collator = DataCollatorForTokenClassification(tokenizer=lowercased_tokenizer)

## Evaluate

In [96]:
seqeval = evaluate.load("seqeval")

In [97]:
label_list = list(label2id.keys())
len(label_list)

71

In [98]:
def compute_metrics(p):
    predictions, labels = p
    # predictions = predictions.logits #it is a tuple with size 1
    np_predictions = np.array(predictions)
    # print(np_predictions.shape)
    # print(predictions[0])
    # print(predictions[1])
    # print(labels)
    predictions = np.argmax(predictions[0], axis=2) #0 picks the first item in the returned outputs [logits, hidden_states]

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Train

In [90]:
tokenized_ebay.set_format("torch")

In [39]:
# device_lib.list_local_devices()

In [120]:
train_dataloader = DataLoader(tokenized_ebay, shuffle=True, batch_size=16, collate_fn = data_collator) #8

In [128]:
# Define a BERT-based NER model with CRF layer
class RoBERTaCRFModel(nn.Module):
    def __init__(self, num_labels, dropout_perc=0.1):
        super(RoBERTaCRFModel, self).__init__()
        self.roberta = AutoModelForTokenClassification.from_pretrained(
                        'numind/entity-recognition-general-sota-v1',
                        output_hidden_states=True, num_labels=num_labels, id2label=id2label, label2id=label2id
                       )
        self.dropout = nn.Dropout(dropout_perc) #regularization, randomly make some entries 0
        self.linear_layer = nn.Linear(self.roberta.config.hidden_size*2, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        emb = torch.cat(
            (outputs.hidden_states[-1], outputs.hidden_states[-7]), #get two hidden states from previous layers
            dim=2
        )
        # emb = outputs.hidden_states[-1]
        pooled_output = self.dropout(emb)
        logits = self.linear_layer(pooled_output)

        # print(labels.shape)
        # print(logits.shape)
        # print(attention_mask)
        # During training
        if labels is not None:
            # Mask out special tokens
            labels[(labels == -100).nonzero(as_tuple=True)[0]] = 70 #make -100 convert to 70
            mask_special_tokens = (labels == 70)

            crf_mask = attention_mask & ~mask_special_tokens

            crf_mask[:, 0] = 1

            # print(logits)
            # print(labels)
            # print(crf_mask)

            # Compute CRF loss, ignoring special tokens
            loss = -self.crf(logits, labels, mask=crf_mask.byte())

            return loss

        # During inference
        else:
            # Mask out special tokens
            crf_mask = attention_mask

            #unmask the first token (always <s>)
            crf_mask[:, 0] = 1

            tags = self.crf.decode(logits, crf_mask.byte())
            return tags

num_labels = 71
model = RoBERTaCRFModel(num_labels)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at numind/entity-recognition-general-sota-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [115]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
pass

In [116]:
# Example training loop with data loader
def train_model(model, train_dataloader, optimizer, num_epochs, device):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        batch_num = 0
        for batch in train_dataloader:
            batch_num += 1
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            loss = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Average Loss: {average_loss}")

In [112]:
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 15
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [139]:
def train_predict_and_write(bs, lr, dropout_perc):
    train_dataloader = DataLoader(tokenized_ebay, shuffle=True, batch_size=bs, collate_fn = data_collator) #8
    num_labels = 71
    model = RoBERTaCRFModel(num_labels, dropout_perc)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=lr)
    num_epochs = 15
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
    )

    train_model(model, train_dataloader, optimizer, num_epochs, device)

    torch.save(model.state_dict(), data_drive+"/models/roberta_crf_model_emb2_tuning_bs" + str(bs) + "_lr" + str(lr))

    model.eval()
    model.to(device)

    res = do_all_predictions(device)

    out = res.loc[res["Aspect Value"]!="null", :]
    out = out.loc[out["Aspect Name"] != "Special", :] #new addition

    return out

In [140]:
# Training loop
#    train_model(model, train_dataloader, optimizer, num_epochs, device)
batch_sizes = [8, 16]
learning_rates = [1e-7, 1e-5, 1e-3]
dropout_percs = [0.01, 0.1]

for batch_size in batch_sizes:
  for learning_rate in learning_rates:
      for perc in dropout_percs:
          print(batch_size, learning_rate, perc)
          out = train_predict_and_write(batch_size, learning_rate, perc)

          filename = "results_1127_roberta_crf_model_emb2_tuning_bs" + str(batch_size) + "_lr" + str(learning_rate) + ".tsv"
          out.to_csv(data_drive + "/results/" + filename, sep="\t", na_rep = "null", header=False, index=False)

8 1e-07 0.01


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at numind/entity-recognition-general-sota-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Average Loss: 102.96547506713867
Epoch 2, Average Loss: 89.76652873535156
Epoch 3, Average Loss: 75.45364001464844
Epoch 4, Average Loss: 67.58077865600586
Epoch 5, Average Loss: 60.91775356140137
Epoch 6, Average Loss: 56.993348864746096
Epoch 7, Average Loss: 53.63264133911133
Epoch 8, Average Loss: 49.464572033691404
Epoch 9, Average Loss: 47.46299756164551
Epoch 10, Average Loss: 47.0711675994873
Epoch 11, Average Loss: 45.42179954071045
Epoch 12, Average Loss: 44.67395586547852
Epoch 13, Average Loss: 43.04060461120606
Epoch 14, Average Loss: 40.90753163452148
Epoch 15, Average Loss: 39.57263053588867
0


RuntimeError: ignored

In [53]:
torch.save(model.state_dict(), data_drive+"/models/roberta_crf_model_emb2_bs16_le1e-5")

## Load model (skip)

In [ ]:
model = RoBERTaCRFModel(num_labels)
load_name = "/models/roberta_crf_model_bs64"
if torch.cuda.is_available():
    model.load_state_dict(torch.load(data_drive + load_name)) #if GPU
else:
    model.load_state_dict(torch.load(data_drive + load_name, map_location=torch.device('cpu'))) #if CPU

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at numind/entity-recognition-general-sota-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Inference on a single sample (test)

In [ ]:
#ideas for improvement - some kind of Autoregression; previous token predicted class impacts next token?
# all of this data is for shoes.

In [55]:
model.eval()
model.to(device)
pass

In [56]:
sample = test_records.loc[test_records["Record Number"] == "6454", "Title"].values[0]
sample

'Minu Damen Schuhe Schnürschuhe 207536 Leder schwarz EUR 42'

In [57]:
lowercased_tokenizer(sample, return_tensors="pt")

{'input_ids': tensor([[    0,  3635,   257,  8234,   225,  1811,   257,   700, 21331,  2768,
          4926, 39037,   700,   291,  2545,  3367,   226, 11736,  8447,  5557,
           329, 10353,  3330,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [68]:
# Inference
model.eval()
with torch.no_grad():
    # Example input for inference
    tokenized_input = lowercased_tokenizer(sample.lower(), return_tensors="pt")
    inference_input_ids = tokenized_input.input_ids.to(device)
    inference_attention_mask = tokenized_input.attention_mask.to(device)

    # During inference, the model returns the decoded sequence of tags
    tags = model(inference_input_ids, attention_mask=inference_attention_mask)

print("Predicted Tags:", tags[0])
print("Predicted Labels:", [id2label[c] for c in tags[0]])

Predicted Tags: [70, 69, 69, 0, 34, 23, 23, 57, 57, 57, 57, 57, 57, 57, 13, 13, 13, 22, 22, 10, 44, 44, 8, 42, 42, 69]
Predicted Labels: ['Special', 'No Tag', 'No Tag', 'Abteilung', 'I-Abteilung', 'Produktart', 'Produktart', 'I-Produktart', 'I-Produktart', 'I-Produktart', 'I-Produktart', 'I-Produktart', 'I-Produktart', 'I-Produktart', 'Herstellernummer', 'Herstellernummer', 'Herstellernummer', 'Obermaterial', 'Obermaterial', 'Farbe', 'I-Farbe', 'I-Farbe', 'EU-Schuhgröße', 'I-EU-Schuhgröße', 'I-EU-Schuhgröße', 'No Tag']


In [59]:
list(zip(lowercased_tokenizer.convert_ids_to_tokens(inference_input_ids[0]), [id2label[c] for c in tags[0]]))


[('<s>', 'Special'),
 ('ĠMin', 'Marke'),
 ('u', 'Abteilung'),
 ('ĠDam', 'Abteilung'),
 ('en', 'I-Abteilung'),
 ('ĠSch', 'Produktart'),
 ('u', 'Produktart'),
 ('he', 'I-Produktart'),
 ('ĠSchn', 'I-Produktart'),
 ('Ã¼', 'I-Produktart'),
 ('rs', 'I-Produktart'),
 ('chu', 'I-Produktart'),
 ('he', 'I-Produktart'),
 ('Ġ20', 'Herstellernummer'),
 ('75', 'Herstellernummer'),
 ('36', 'I-Herstellernummer'),
 ('ĠL', 'No Tag'),
 ('eder', 'Farbe'),
 ('Ġsch', 'Farbe'),
 ('war', 'I-Farbe'),
 ('z', 'I-Farbe'),
 ('ĠEUR', 'No Tag'),
 ('Ġ42', 'EU-Schuhgröße'),
 ('</s>', 'No Tag')]

## Case Outputs Functionality

In [138]:
def similarity_score(word1, word2):
    # Calculate the Levenshtein distance
    levenshtein_distance = distance.levenshtein(word1, word2)

    # Calculate the similarity score (normalized)
    max_length = max(len(word1), len(word2))
    similarity = 1 - (levenshtein_distance / max_length)

    return similarity

In [137]:
def get_most_similar_word(text, word):
    #problem: EUR 42,5 when word = "eur 42 , 5"

    if " " in word:
        #check1: try to see by subtoken
        subtoken1 = word.split(" ")[0] #first subtoken in sequence
        start_index = text.find(get_most_similar_word(text, subtoken1))
        subtoken2 = word.split(" ")[-1] #last subtoken
        end_sim_word = get_most_similar_word(text, subtoken2)
        end_index = text.find(end_sim_word) + len(end_sim_word) #go till the end
        if start_index != -1 and start_index < end_index: #if not, just get most similar word
            return text[start_index:end_index]

    #check2: check a space-separated word
    words = pd.Series(text.split(" "))
    scores1 = words.apply(lambda x: similarity_score(x, word)) #score if lowercase word
    scores2 = words.apply(lambda x: similarity_score(x, word.upper())) #score if uppercase word
    scores3 = words.apply(lambda x: similarity_score(x, word[0].upper() + word[1:]))
    scores = pd.concat([scores1, scores2],axis=1).max(axis=1)
    return words[scores.argmax()]

In [136]:
def case_outputs(outputs, text):
    lower_text = text.lower()

    for i in range(len(outputs)):
        token = outputs[i][1]
        subtoken = token.split(" ")[0] #consider only the first item in a multi-token
        start_index = lower_text.find(subtoken)
        last_subtoken = token.split(" ")[-1]
        end_index = lower_text.find(last_subtoken) + len(last_subtoken)
        cased_token = text[start_index:end_index]


        if start_index == -1 or start_index > end_index or cased_token == "": #not found
            outputs[i][1] = get_most_similar_word(text, token)
            continue

        outputs[i][1] = cased_token

    return outputs

In [135]:
#function to obtain tag predictions and merge aspect values
def process_title(title, device, gpu=True):
    inputs = lowercased_tokenizer(title.lower(), return_tensors="pt")

    model.eval()
    with torch.no_grad():
        # Example input for inference
        inference_input_ids = inputs.input_ids.to(device)
        inference_attention_mask = inputs.attention_mask.to(device)

        # During inference, the model returns the decoded sequence of tags
        tags = model(inference_input_ids, attention_mask=inference_attention_mask)

    predicted_token_class = [id2label[c] for c in tags[0]]
    decoded = lowercased_tokenizer.convert_ids_to_tokens(inference_input_ids[0])

    # Merge I-<Tag> with <Tag>
    #Need to implement: checking previous token was the same tag, majority voting, ##
    merged_tokens = [] # list of [tag, token sequence]
    prev_tag, prev_token = "", ""
    current_tag = None

    # print(list(zip(decoded, predicted_token_class)))
    for token, tag in zip(decoded, predicted_token_class):
      # exclude CLS and SEP
      if token in ['<s>', '</s>']:
          continue

      if not token.startswith("Ġ"): #existing token token
          if len(merged_tokens) > 0:
              merged_tokens[-1][1] += token #add to previous word
              continue  # doing no majority voting, go to the next word - THIS IS A GOOD THING FOR THIS MODEL
          else:
              token = token[1:]  # first observed token, act as if this subword is a complete word

      if tag.startswith('I-'):
          tag = tag[2:]
          # Check if the current "I-" tag matches the previous entry's tag
          if current_tag is not None and tag == current_tag:
              merged_tokens[-1][1] += ' ' + token
          else:
              merged_tokens.append([tag, token])
          current_tag = tag
          continue

      current_tag = tag
      merged_tokens.append([current_tag, token])

    # print(merged_tokens)
    # return merged_tokens
    return case_outputs(merged_tokens, title) # returns cased and most similar outputs
    # find a way to avoid casing?


In [64]:
sample

'Minu Damen Schuhe Schnürschuhe 207536 Leder schwarz EUR 42'

In [65]:
list(zip(lowercased_tokenizer.convert_ids_to_tokens(inference_input_ids[0]), [id2label[c] for c in tags[0]]))


[('<s>', 'Special'),
 ('ĠMin', 'Marke'),
 ('u', 'Abteilung'),
 ('ĠDam', 'Abteilung'),
 ('en', 'I-Abteilung'),
 ('ĠSch', 'Produktart'),
 ('u', 'Produktart'),
 ('he', 'I-Produktart'),
 ('ĠSchn', 'I-Produktart'),
 ('Ã¼', 'I-Produktart'),
 ('rs', 'I-Produktart'),
 ('chu', 'I-Produktart'),
 ('he', 'I-Produktart'),
 ('Ġ20', 'Herstellernummer'),
 ('75', 'Herstellernummer'),
 ('36', 'I-Herstellernummer'),
 ('ĠL', 'No Tag'),
 ('eder', 'Farbe'),
 ('Ġsch', 'Farbe'),
 ('war', 'I-Farbe'),
 ('z', 'I-Farbe'),
 ('ĠEUR', 'No Tag'),
 ('Ġ42', 'EU-Schuhgröße'),
 ('</s>', 'No Tag')]

In [66]:
process_title(sample, gpu=True)

[['No Tag', 'Minu'],
 ['Abteilung', 'Damen'],
 ['Produktart', 'Schuhe Schnürschuhe'],
 ['Herstellernummer', '207536'],
 ['Obermaterial', 'Leder'],
 ['Farbe', 'schwarz'],
 ['EU-Schuhgröße', 'EUR 42']]

## Do prediction on test dataset


In [134]:
def do_all_predictions(device):
    result_list = []
    count = 0
    for index, row in test_records.iterrows():
        if count % 1000 == 0: #1000
          print(count)
        record_number = row['Record Number']
        title = row['Title']
        tagged_tokens = process_title(title, device, gpu=True)

        for tag, token_sequence in tagged_tokens:
            result_list.append({
                'Record Number': record_number,
                'Aspect Name': tag,
                'Aspect Value': token_sequence
            })
        count += 1

    result = pd.DataFrame(result_list)
    result = result.loc[(result["Aspect Name"] != "No Tag")&(result["Aspect Name"] != "Obscure"), :] # remove No Tag, obscure
    result["Record Number"] = result["Record Number"].astype(int)

    return result

In [72]:
# test_records.loc[test_records["Record Number"] == "10030", "Title"].values #check Women's --> i'm pretty sure it breaks

array(['Adidas Super Light Loop Runner Black White Mens Trainers - C77018'],
      dtype=object)

In [ ]:
res = do_all_predictions()

In [79]:
res.loc[result["Record Number"] == 10030,:]

,Record Number,Aspect Name,Aspect Value
46474,10030,Marke,Adidas
46475,10030,Produktlinie,Super
46476,10030,Modell,Light Loop
46477,10030,Produktart,Runner
46478,10030,Farbe,Black White
46479,10030,Abteilung,Mens
46480,10030,Produktart,Trainers
46482,10030,Herstellernummer,C77018


In [80]:
# check to make sure no NaN values
np.sum(res.isna())

Record Number    0
Aspect Name      0
Aspect Value     0
dtype: int64

In [81]:
res.loc[(res["Aspect Name"]=="No Tag")|(res["Aspect Name"]=="Obscure")|(res["Aspect Name"]=="Special"), :]

,Record Number,Aspect Name,Aspect Value


In [82]:
res.loc[res["Aspect Value"]=="null", :] #still values to fix, but let's test for now
# CHECK NA REP: 11233, 11817

,Record Number,Aspect Name,Aspect Value
57540,11233,Modell,null


In [83]:
len(res)

153532

In [84]:
out = res.loc[res["Aspect Value"]!="null", :]
out = out.loc[out["Aspect Name"] != "Special", :] #new addition
out

,Record Number,Aspect Name,Aspect Value
0,5001,Marke,NIKE
1,5001,Produktlinie,FREE
2,5001,Modell,RUN 3 SHIELD 5.0
3,5001,Stil,SNEAKERS
4,5001,Produktart,LAUFSCHUHE
...,...,...,...
230695,30000,Marke,Puma
230696,30000,Produktart,Turnschuhe
230697,30000,Farbe,weiß
230698,30000,Abteilung,unisex


In [ ]:
#POST PROCESSING FOR "SPECIAL" CLASSIFICATION REQUIRED

In [85]:
# save to file
filename = "results_1127_roberta_crf_emb2.tsv"
out.to_csv(data_drive + "/results/" + filename, sep="\t", na_rep = "null", header=False, index=False)

## Tests (skip)

In [ ]:
#testing the I-tag fix

text = train_df.Title[0]
inputs = tokenizer(text, return_tensors="pt")
cuda_in = inputs.input_ids.to("cuda")
decoded = tokenizer.convert_ids_to_tokens(cuda_in[0])
model.eval() #not sure if this is correct
with torch.no_grad():
    logits = model(cuda_in).logits
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
dict(zip(decoded, predicted_token_class))


merged_tokens = [] # list of [tag, token sequence]
prev_tag, prev_token = "", ""
current_tag = None

for token, tag in zip(decoded, predicted_token_class):
    if token.startswith("##"):
        if len(merged_tokens) > 0:
            merged_tokens[-1][1] += token[2:]
            continue  # doing no majority voting, go to the next word
        else:
            token = token[2:]  # first observed token, act as if this subword is a complete word

    if tag.startswith('I-'):
        tag = tag[2:]
        # Check if the current "I-" tag matches the previous entry's tag
        if current_tag is not None and tag == current_tag:
            merged_tokens[-1][1] += ' ' + token
        else:
            merged_tokens.append([tag, token])
        current_tag = tag
        continue

    if tag != 'O' and token not in ['[CLS]', '[SEP]']:
        current_tag = tag
        merged_tokens.append([current_tag, token])
    else:
        current_tag = None

print(merged_tokens)


In [ ]:
ex_title = train_df.loc[0, "Title"]
ex_title

In [ ]:
tokenizer(ex_title)

In [ ]:
merged_tokens = [] # list of [tag, token sequence]
prev_tag, prev_token = "", ""
current_tag = None

for token, tag in zip(decoded, predicted_token_class):
    if token.startswith("##"):
        if len(merged_tokens) > 0:
            merged_tokens[-1][1] += token[2:]
            continue  # doing no majority voting, go to the next word
        else:
            token = token[2:]  # first observed token, act as if this subword is a complete word

    if tag.startswith('I-'):
        tag = tag[2:]
        # Check if the current "I-" tag matches the previous entry's tag
        if current_tag is not None and tag == current_tag:
            merged_tokens[-1][1] += ' ' + token
        else:
            merged_tokens.append([tag, token])
        current_tag = tag
        continue

    if tag != 'O' and token not in ['[CLS]', '[SEP]']:
        current_tag = tag
        merged_tokens.append([current_tag, token])
    else:
        current_tag = None

print(merged_tokens)
